### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [1]:
!pip install pyspark==3.1.3
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 MB 2.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 7.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.3-py2.py3-none-any.whl size=214463469 sha256=1dc3423fd73b29ba36f2ad0733c2afdd436cafd711aa56bfacf8317b7b923d37
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/ad/8e/49/44c110bb8e008d0778c6577d600d46047c6478ecca3f8f1517
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.4.4
    Uninstalling pyspark-3.4.4:
      Successfully uninstalled pyspark-3.4.4


In [ ]:
# Start session

In [25]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler

In [3]:
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

24/11/25 15:33:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [6]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv"
response = requests.get(url)

# Guardar el archivo localmente
with open("searchterms.csv", "wb") as file:
    file.write(response.content)
print("File downloaded successfully.")

Archivo descargado con éxito.


In [ ]:
# Load the csv into a spark dataframe

In [7]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)
df.show()

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021|        laptop|
| 12|   11|2021|        laptop|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| gaming laptop|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|     mobile 5g|
| 12|   11|2021|        laptop|
+---+-----+----+--------------+
only showing top 20 rows



In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [9]:
num_rows = df.count()
num_cols = len(df.columns)

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 10000
Number of columns: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [10]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [11]:
df.printSchema()

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [12]:
df.filter(df["searchterm"] == 'gaming laptop').count()

499

In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [14]:
df.groupBy("searchterm").count().orderBy("count", ascending=False).show(5)

[Stage 17:=================================================>   (185 + 10) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+
only showing top 5 rows



In [18]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

from pyspark.ml.regression import LinearRegressionModel

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz"
response = requests.get(url)

# Guardar el archivo localmente
with open("model.tar.gz", "wb") as file:
    file.write(response.content)
print("File downloaded successfully.")

File downloaded successfully.


In [20]:
import tarfile

# Extraer el archivo
with tarfile.open("model.tar.gz", "r:gz") as tar:
    tar.extractall("model")
    print("Model extracted to 'model/' directory")

Model extracted to 'model/' directory


In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [22]:
model = LinearRegressionModel.load('model/sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [27]:
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','sales')
    predictions = model.transform(__)
    predictions.select('prediction').show()
predict(2023)


+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



24/11/25 16:57:17 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/11/25 16:57:17 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
